<a href="https://colab.research.google.com/github/JonasZimmer1994/deeplearning_wavenet/blob/master/wavenet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!pip3 install http://download.pytorch.org/whl/cu80/torch-0.3.0.post4-cp36-cp36m-linux_x86_64.whl 

     |████████████████████████████████| 592.3MB 50.2MB/s 
ERROR: torchvision 0.4.2 has requirement torch==1.3.1, but you'll have torch 0.3.0.post4 which is incompatible.
ERROR: fastai 1.0.59 has requirement torch>=1.0.0, but you'll have torch 0.3.0.post4 which is incompatible.
  Found existing installation: torch 0.4.1
    Uninstalling torch-0.4.1:
      Successfully uninstalled torch-0.4.1


In [2]:
!git clone https://github.com/vincentherrmann/pytorch-wavenet.git

Cloning into 'pytorch-wavenet'...
remote: Enumerating objects: 1154, done.
remote: Total 1154 (delta 0), reused 0 (delta 0), pack-reused 1154
Receiving objects: 100% (1154/1154), 268.94 MiB | 32.59 MiB/s, done.
Resolving deltas: 100% (713/713), done.


In [3]:
%cd /content/pytorch-wavenet/

/content/pytorch-wavenet


In [4]:
!pip install tensorflow==1.12.0
import tensorflow as tf
print(tf.__version__)

     |████████████████████████████████| 83.1MB 126kB/s 
     |████████████████████████████████| 3.1MB 35.6MB/s 
  Found existing installation: tensorboard 1.15.0
    Uninstalling tensorboard-1.15.0:
      Successfully uninstalled tensorboard-1.15.0
  Found existing installation: tensorflow 1.15.0
    Uninstalling tensorflow-1.15.0:
      Successfully uninstalled tensorflow-1.15.0


/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or 

1.12.0


In [0]:
import torch
from wavenet_model import *
from audio_data import WavenetDataset
from wavenet_training import *
from model_logging import *

In [8]:
# initialize cuda option
dtype = torch.FloatTensor # data type
ltype = torch.LongTensor # label type

use_cuda = torch.cuda.is_available()
if use_cuda:
    print('use gpu')
    dtype = torch.cuda.FloatTensor
    ltype = torch.cuda.LongTensor

use gpu


In [9]:
model = WaveNetModel(layers=10,
                     blocks=3,
                     dilation_channels=32,
                     residual_channels=32,
                     skip_channels=1024,
                     end_channels=512, 
                     output_length=16,
                     dtype=dtype, 
                     bias=True)
# model = load_latest_model_from('snapshots', use_cuda=use_cuda)

print('model: ', model)
print('receptive field: ', model.receptive_field)
print('parameter count: ', model.parameter_count())

model:  WaveNetModel(
  (filter_convs): ModuleList(
    (0): Conv1d (32, 32, kernel_size=(2,), stride=(1,))
    (1): Conv1d (32, 32, kernel_size=(2,), stride=(1,))
    (2): Conv1d (32, 32, kernel_size=(2,), stride=(1,))
    (3): Conv1d (32, 32, kernel_size=(2,), stride=(1,))
    (4): Conv1d (32, 32, kernel_size=(2,), stride=(1,))
    (5): Conv1d (32, 32, kernel_size=(2,), stride=(1,))
    (6): Conv1d (32, 32, kernel_size=(2,), stride=(1,))
    (7): Conv1d (32, 32, kernel_size=(2,), stride=(1,))
    (8): Conv1d (32, 32, kernel_size=(2,), stride=(1,))
    (9): Conv1d (32, 32, kernel_size=(2,), stride=(1,))
    (10): Conv1d (32, 32, kernel_size=(2,), stride=(1,))
    (11): Conv1d (32, 32, kernel_size=(2,), stride=(1,))
    (12): Conv1d (32, 32, kernel_size=(2,), stride=(1,))
    (13): Conv1d (32, 32, kernel_size=(2,), stride=(1,))
    (14): Conv1d (32, 32, kernel_size=(2,), stride=(1,))
    (15): Conv1d (32, 32, kernel_size=(2,), stride=(1,))
    (16): Conv1d (32, 32, kernel_size=(2,), st

In [10]:
data = WavenetDataset(dataset_file='train_samples/bach_chaconne/dataset.npz',
                      item_length=model.receptive_field + model.output_length - 1,
                      target_length=model.output_length,
                      file_location='train_samples/bach_chaconne',
                      test_stride=500)
print('the dataset has ' + str(len(data)) + ' items')

one hot input
the dataset has 598277 items


In [0]:
def generate_and_log_samples(step):
    sample_length=32000
    gen_model = load_latest_model_from('snapshots', use_cuda=False)
    print("start generating...")
    samples = generate_audio(gen_model,
                             length=sample_length,
                             temperatures=[0.5])
    tf_samples = tf.convert_to_tensor(samples, dtype=tf.float32)
    logger.audio_summary('temperature_0.5', tf_samples, step, sr=16000)

    samples = generate_audio(gen_model,
                             length=sample_length,
                             temperatures=[1.])
    tf_samples = tf.convert_to_tensor(samples, dtype=tf.float32)
    logger.audio_summary('temperature_1.0', tf_samples, step, sr=16000)
    print("audio clips generated")

In [0]:
logger = TensorboardLogger(log_interval=200,
                           validation_interval=400,
                           generate_interval=1000,
                           generate_function=generate_and_log_samples,
                           log_dir="logs/chaconne_model")

In [0]:
trainer = WavenetTrainer(model=model.cuda(),
                         dataset=data,
                         lr=0.001,
                         snapshot_path='snapshots',
                         snapshot_name='chaconne_model',
                         snapshot_interval=1000,
                         #logger=logger,
                         dtype=dtype,
                         ltype=ltype)

print('start training...')
trainer.train(batch_size=16,
              epochs=10)

start training...
epoch 0
loss at step 50: 5.227417221069336
